In [1]:
import os
import pandas as pd
import torch
import numpy as np
import json
from sklearn.preprocessing import StandardScaler
from itertools import islice

In [9]:
# 定義 CSV 文件所在的資料夾路徑
folder_path = '/home/xcliu/AS/CutUSGSignal'

# 初始化一個空的 DataFrame 來儲存合併後的數據
combined_df = pd.DataFrame()

# 遍歷資料夾中的所有 CSV 文件
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # 讀取每個 CSV 文件並將其追加到 DataFrame 中
        df = pd.read_csv(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# 儲存合併後的 DataFrame 為 PKL 文件
combined_df.to_pickle('usg_data.pkl')

print("合併並儲存完成!")

合併並儲存完成!


In [10]:
usg_data = pd.read_pickle('/home/xcliu/AS/as_pm/usg_data.pkl')

In [11]:
# 定义均值和标准差
scaler_filepath = os.path.join("./data/scaler", 'scaler_info.json')
with open(scaler_filepath, "r") as json_file:
    scaler_info = json.load(json_file)  
mean = scaler_info['mean']
std_dev = scaler_info['std']

# 对数据进行标准化
usg_data['USGSignal_scaled'] = (usg_data[['USGSignal']].values - mean) / std_dev

# 如果你想要保存标准化后的数据，可以这样做：
# usg_data_standardized.to_pickle('/path/to/save/standardized_usg_data.pkl')

# print(usg_data_standardized.head())  # 查看标准化后的数据


In [33]:
grouped = usg_data.groupby(['filename', 'Group'])

output_dir = 'data'  # 指定存儲 JSON 文件的目錄
# os.makedirs(output_dir, exist_ok=True)  # 創建目錄（如果不存在的話）

# 初始化標準化對象
scaler = StandardScaler()

# 先對所有數據的 'USGSignal' 進行標準化
usg_signals = usg_data[['USGSignal']].values
usg_signals_scaled = scaler.fit_transform(usg_signals)

# 用標準化的數據更新原始數據
usg_data['USGSignal_scaled'] = usg_signals_scaled

# 獲取 scaler 的 mean 和 std
scaler_mean = scaler.mean_[0]
scaler_std = scaler.scale_[0]

# 將 mean 和 std 儲存為 JSON 文件，方便日後返回原數據
scaler_info = {
    'mean': scaler_mean,
    'std': scaler_std
}
scaler_filepath = os.path.join(output_dir, 'scaler_info.json')
with open(scaler_filepath, 'w') as json_file:
    json.dump(scaler_info, json_file)

In [12]:
grouped = usg_data.groupby(['filename', 'Group'])

output_dir = 'data/validation'  # 指定存儲 JSON 文件的目錄
# os.makedirs(output_dir, exist_ok=True)  # 創建目錄（如果不存在的話）

life_time = 1
for (seq_id, sub_seq_id), group in grouped:
    sequence_features = group[['USGSignal_scaled']].values.flatten().tolist()
    label = life_time / 9000  # 900k

    # sequence_tensor = torch.tensor(sequence_features, dtype=torch.float32)
    # label_tensor = torch.tensor(label, dtype=torch.float32)
    
    # # Convert tensors to lists and scalars
    # sequence_data = sequence_tensor.tolist()
    # label_data = label_tensor.item()
    
    # Create a dictionary to store sequence and label data
    data = {
        'sequence': sequence_features,
        'label': label
    }
    
    # Create a filename for each group
    filename = f"cap3_{seq_id}_{sub_seq_id}.json"
    filepath = os.path.join(output_dir, filename)
    
    # Save to JSON file
    with open(filepath, 'w') as json_file:
        json.dump(data, json_file)
    
    life_time += 2


In [ ]:
grouped = usg_data.groupby(['filename', 'Group'])

for (seq_id, sub_seq_id), group in grouped:
    print(f"Group: sequence_id={seq_id}, sub_sequence_id={sub_seq_id}")

# sequences = []
# labels = []
# life_time = 1
# for (seq_id, sub_seq_id), group in grouped:
#     sequence_features = group[['USGSignal']].values.flatten().tolist()
#     label = life_time / 9000  # 900k

#     sequence_tensor = torch.tensor(sequence_features, dtype=torch.float32)
#     label_tensor = torch.tensor(label, dtype=torch.float32)

    
#     # sequences.append(sequence_tensor.tolist())
#     # labels.append(label_tensor.tolist())
    
#     life_time += 2

## 檢查是否有sequence

In [5]:
# 定义要检查的目录
data_dir = "./data"  # 替换为你的目录路径

# 遍历目录下的所有文件
for filename in os.listdir(data_dir):
    if filename.endswith(".json"):
        filepath = os.path.join(data_dir, filename)
        try:
            # 打开并加载JSON文件
            with open(filepath, 'r') as file:
                data = json.load(file)
            
            # 检查' sequence'是否在JSON数据中
            if 'sequence' not in data:
                print(f"'sequence' key not found in: {filename}")
        
        except json.JSONDecodeError as e:
            print(f"Error reading {filename}: {e}")


## 移動

In [7]:
import os
import shutil

# 定义源目录和目标目录
source_dir = "./data"
target_dir = os.path.join(source_dir, "training")

# 如果目标目录不存在，则创建它
# if not os.path.exists(target_dir):
#     os.makedirs(target_dir)

# 遍历源目录下的所有文件
for filename in os.listdir(source_dir):
    if filename.endswith(".json"):
        # 构建完整的源文件路径和目标文件路径
        source_path = os.path.join(source_dir, filename)
        target_path = os.path.join(target_dir, filename)
        
        # 移动文件到目标目录
        shutil.move(source_path, target_path)
        # print(f"Moved: {filename} to {target_dir}")


## 改名

In [1]:
import os

# 定义目标目录
target_dir = "./data/training"

# 遍历目标目录下的所有文件
for filename in os.listdir(target_dir):
    if filename.endswith(".json"):
        # 构建完整的旧文件路径和新文件路径
        old_filepath = os.path.join(target_dir, filename)
        new_filename = "cap1_" + filename
        new_filepath = os.path.join(target_dir, new_filename)
        
        # 重命名文件
        os.rename(old_filepath, new_filepath)
        # print(f"Renamed: {filename} to {new_filename}")


In [8]:
# Calculate mean and standard deviation
mean = np.mean(usg_data['USGSignal'])
std_dev = np.std(usg_data['USGSignal'])

print(f"Mean: {mean}")
print(f"Standard Deviation: {std_dev}")

# Optionally, save these values to a file
scaler_params = {
    'mean': mean,
    'std_dev': std_dev
}

# with open('scaler_params.json', 'w') as file:
#     json.dump(scaler_params, file)


Mean: 0.0669816558396192
Standard Deviation: 0.16946513850743108


In [9]:
scaler_params

{'mean': np.float64(0.0669816558396192),
 'std_dev': np.float64(0.16946513850743108)}

In [13]:
usg_data = {
    "sequences": sequences,
    "labels": labels
}

# 保存為 JSON 文件


In [5]:
# 保存 sequences 和 labels
# torch.save({'sequences': sequences, 'labels': labels}, 'usg_data.pth')

# 读取数据
loaded_data = torch.load('usg_data.pth')
sequences = loaded_data['sequences']
torch.save(sequences, 'sequences.pth')
torch.save(labels, 'label.pth')
labels = loaded_data['labels']

/tmp/ipykernel_260430/727725011.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load('usg_data.pth')


In [6]:
print(sequences[0])
print(labels[0])

tensor([ 0.2457,  0.2118,  0.1550, -0.0696, -0.0562,  0.1987,  0.1761,  0.1581,
        -0.0531, -0.0971, -0.0983, -0.1187, -0.1477, -0.1175, -0.0857, -0.0806,
         0.1782,  0.2008,  0.2011,  0.2548,  0.2307,  0.1956,  0.1502, -0.1013,
        -0.1410, -0.1535, -0.1477, -0.1892, -0.1492, -0.1041, -0.0571,  0.2307,
         0.2383,  0.2447,  0.2341,  0.2600,  0.2100,  0.1636, -0.0668, -0.1099,
        -0.2103, -0.1956, -0.1828, -0.2026, -0.1636, -0.1059,  0.1883,  0.2673,
         0.2942,  0.2884,  0.2649,  0.2615,  0.2210,  0.1535, -0.1126, -0.1620,
        -0.2332, -0.2362, -0.2133, -0.1810, -0.1730, -0.1041,  0.1901,  0.2344,
         0.2756,  0.3418,  0.3253,  0.2972,  0.2469,  0.2210,  0.1532, -0.1068,
        -0.1685, -0.2042, -0.2698, -0.2679, -0.2399, -0.1913, -0.1700, -0.0885,
         0.1724,  0.2426,  0.2927,  0.3220,  0.3339,  0.3079,  0.2692,  0.2197,
         0.1541, -0.0958, -0.1385, -0.1639, -0.1703, -0.1620, -0.1428, -0.1056,
        -0.0635,  0.1834,  0.2167,  0.23